In [5]:
from parse import parse, compile
# import re

In [6]:
def process_input(_input):
    polymer_template, pair_insertion_rules = _input.split('\n\n')
    
    p = compile("{} -> {}")
    pair_insertion_rules = { p.parse(rule)[0]:p.parse(rule)[1] for rule in  pair_insertion_rules.split('\n') }
    pair_insertion_rules = { k:(k[0]+v,v+k[1]) for k,v in pair_insertion_rules.items() }
    return polymer_template, pair_insertion_rules

with open('./input.txt', 'r') as f:
    prod = process_input(f.read())
    
with open('./test_input.txt', 'r') as f:
    test = process_input(f.read())

### Successful Attempt

In [7]:
def polymerization(pairs, rules):
    next_pairs = { k:0 for k in rules.keys() }
    for pair, value in pairs.items():
        if value > 0:
            next_pairs[rules[pair][0]] += value
            next_pairs[rules[pair][1]] += value
    return next_pairs

def answer(char_frequency):
    # chars are counted twice when in pair form
    # odd counts are on the edges
    char_frequency = {k:v//2+v%2 for k,v in char_frequency.items()}
    return max(char_frequency.values()) - min(char_frequency.values())

def main(_input, steps=10):
    template, rules = _input
    pairs = { k:0 for k in rules.keys() }
    for pair in list(zip(template[:-1],template[1:])):
        pairs[''.join(pair)] += 1

    for _ in range(steps):
        pairs = polymerization(pairs, rules)

    chars = set()
    for rule in rules.keys():
        chars = chars | set(rule)

    char_frequency = { c:0 for c in chars }

    for pair, value in pairs.items():
        char_frequency[pair[0]] += value
        char_frequency[pair[1]] += value
        
    return answer(char_frequency)

print("Part 1", main(prod))
print("Part 2", main(prod, 40))

Part 1 3058
Part 2 3447389044530


### Attempt 1

In [4]:
def polymerization(template, rules):
    polymer = ''
    for index in range(len(template) - 1):
        polymer += template[index] + rules.get(template[index] + template[index+1], '')
    polymer += template[-1]
    return polymer

def main():
    template, rules = prod
    for _ in range(15):
        template = polymerization(template, rules)

    counter = Counter(template).most_common()
    return counter[0][1] - counter[-1][1]

### Attempt 2

In [91]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None
        
class LinkedList:
    """Broke after refactoring input for a different approach."""
    def __init__(self, nodes=None):
        self.head = None
        if nodes is not None:
            node = Node(data=nodes.pop(0))
            self.head = node
            for n in nodes:
                node.next = Node(data=n)
                node = node.next
    
    def __str__(self): 
        s = ''
        node = self.head
        while node is not None:
            s += node.data
            node = node.next
        return s
                
def polymerization(linked_list, rules):
    node = linked_list.head
    while node.next is not None:
        if node.data + node.next.data in rules:
            insert = Node(rules[node.data + node.next.data])
            insert.next = node.next
            node.next = insert
            node = insert.next
        else:
            node = node.next
            
def main():
    template, rules = test
    linked_list = LinkedList(list(template))
    for _ in range(40):
        polymerization(linked_list, rules)

    s = str(linked_list)
    counter = Counter(s).most_common()
    return counter[0][1] - counter[-1][1]

### Attempt 3

In [20]:
# https://docs.python.org/3/library/re.html#re.sub

match = {'CH': 'CBH',
         'HH': 'HNH',
         'CB': 'CHB',
         'NH': 'NCH',
         'HB': 'HCB',
         'HC': 'HBC',
         'HN': 'HCN',
         'NN': 'NCN',
         'BH': 'BHH',
         'NC': 'NBC',
         'NB': 'NBB',
         'BN': 'BBN',
         'BB': 'BNB',
         'BC': 'BBC',
         'CC': 'CNC',
         'CN': 'CCN'}

def replace(matchobj):
    return match.get(matchobj.group(0), matchobj.group(0))
re.sub('|'.join(test[1].keys()), replace, 'NNCB')

'NCNCHB'